# 1. Basics of working with candles data and indicators

We have two ways how to get candles data:

1. Directly from ByBit (slow performance, suitable for small data sets)
2. From local database (fast performance, need to perform data sync first)

To run this example, make sure that you:

* Start up your PostgreSQL instance:

```bash
docker-compose up -d db
```

* Apply migrations and sync candles data (e.g. through IPython):

```python
from app import db

await db.start()
await db.apply_migrations()
await db.sync_candles_from_bybit()
```

At first, we need to define some constraints of dataset, which we will operate:

In [1]:
import datetime as dt

from app.core import Timeframe


start_at = dt.datetime(2022, 8, 1)
end_at = dt.datetime(2022, 11, 1)
asset = 'BTCUSDT'
timeframe = Timeframe.H1

Let's get candles from ByBit. It's simple, just import the client and call `get_candles` method:

In [2]:
from app import bybit

bybit_candles = await bybit.client.get_candles(asset, timeframe, start_at, end_at)

[00:47:32.322] Downloading candles for BTCUSDT[1h]...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2208/2208 [00:02<00:00, 903.83it/s]


Make simple check of our data: 

In [3]:
print('Total:', len(bybit_candles))
print('Min:', bybit_candles[0])
print('Max:', bybit_candles[-1])

Total: 2208
Min: {'open_time': datetime.datetime(2022, 8, 1, 0, 0), 'open': 23739.5, 'high': 23967.0, 'low': 23693.5, 'close': 23827.0, 'volume': 6686.139}
Max: {'open_time': datetime.datetime(2022, 10, 31, 23, 0), 'open': 20365.0, 'high': 20455.5, 'low': 20350.0, 'close': 20419.5, 'volume': 3354.755}


To get candles from DB, we have the same interface as in case of Bybit. But first of that, we need to initialize DB connection through `db.start`:

In [4]:
from app import db

await db.start()

db_candles = await db.get_candles(asset, timeframe, start_at, end_at)

Make same checks that we made for Bybit dataset:

In [5]:
print('Total:', len(db_candles))
print('Min:', db_candles[0])
print('Max:', db_candles[-1])

Total: 2208
Min: {'open_time': datetime.datetime(2022, 8, 1, 0, 0), 'open': 23739.5, 'close': 23827.0, 'low': 23693.5, 'high': 23967.0, 'volume': 6686.139}
Max: {'open_time': datetime.datetime(2022, 10, 31, 23, 0), 'open': 20365.0, 'close': 20419.5, 'low': 20350.0, 'high': 20455.5, 'volume': 3354.755}


Now it's time to pass data to indicators.

The main goal of indicators design - make them as simple as possible with gaining max performance of native Python (and Numpy in some calculations). They are providing only loop-based interface, no vector calculations. As a result, indicators are less error-prone because we can use the same code for every trading needs (backtesting & live trading). The main drawback - restricted performance

In [6]:
from app.indicators import *
from app.config import logger

# Use tqdm to check calc performance
from tqdm import tqdm


# Three different type of indicators with the 200-ticks moving window (ticks based on timefrme of dataset, in our case - 5 minutes)
sma = SMA(20)
ema = EMA(20)
wma = WMA(20)

sma_signals = {}
ema_signals = {}
wma_signals = {}

# We can build dataset of indicator output if we need, indicator does not made it by themself

logger.info('Calculating SMA...')
for candle in tqdm(db_candles):
    sma_signals[candle['open_time']] = sma.calculate(candle['close'])


logger.info('Calculating EMA...')
for candle in tqdm(db_candles):
    ema_signals[candle['open_time']] = ema.calculate(candle['close'])
    

logger.info('Calculating WMA...')
for candle in tqdm(db_candles):
    wma_signals[candle['open_time']] = wma.calculate(candle['close'])
    
    
print()
print(f'Last SMA: {list(sma_signals.items())[-2:]}')
print(f'Last EMA: {list(ema_signals.items())[-2:]}')
print(f'Last WMA: {list(wma_signals.items())[-2:]}')

[00:47:35.585] Calculating SMA...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2208/2208 [00:00<00:00, 777779.73it/s]

[00:47:35.590] Calculating EMA...



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2208/2208 [00:00<00:00, 1246939.98it/s]

[00:47:35.594] Calculating WMA...



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2208/2208 [00:00<00:00, 47704.04it/s]


Last SMA: [(datetime.datetime(2022, 10, 31, 22, 0), 20522.175), (datetime.datetime(2022, 10, 31, 23, 0), 20514.2)]
Last EMA: [(datetime.datetime(2022, 10, 31, 22, 0), 20519.278377571918), (datetime.datetime(2022, 10, 31, 23, 0), 20509.77947602707)]
Last WMA: [(datetime.datetime(2022, 10, 31, 22, 0), 20498.69523809524), (datetime.datetime(2022, 10, 31, 23, 0), 20488.916666666668)]
